# Análise do Fluxo de Histórias

Notebook para o auxílio da análise das histórias do chatbot


## Instalação

### Configurando jupyter

In [1]:
%matplotlib inline

import logging, io, json, warnings
logging.basicConfig(level="INFO")
warnings.filterwarnings('ignore')

def pprint(o):
    # small helper to make dict dumps a bit prettier
    print(json.dumps(o, indent=2))

### Instalação do rasa

Intalando as configurações necessárias a partir do arquivo requirements.txt

In [2]:
import sys
python = sys.executable

# In your environment run:
!{python} -m pip install -r requirements.txt


  Using cached https://files.pythonhosted.org/packages/ee/3d/b25d35a9f0d381dd1c02d8e04b37c353caaaff4bc32150328eeebe4931f5/prompt_toolkit-1.0.14-py3-none-any.whl


jupyter-console 6.0.0 has requirement prompt-toolkit<2.1.0,>=2.0.0, but you'll have prompt-toolkit 1.0.14 which is incompatible.
ipython 7.1.1 has requirement prompt-toolkit<2.1.0,>=2.0.0, but you'll have prompt-toolkit 1.0.14 which is incompatible.
  Found existing installation: prompt-toolkit 2.0.7
    Uninstalling prompt-toolkit-2.0.7:
      Successfully uninstalled prompt-toolkit-2.0.7


Verificando versões:

In [3]:
import rasa_nlu
import rasa_core

print("rasa_nlu: {} rasa_core: {}".format(rasa_nlu.__version__, rasa_core.__version__))


rasa_nlu: 0.13.5 rasa_core: 0.11.12


### Outras ferramentas necessárias
Para algumas visualizações é necessário graphviz.

Instalando:

In [4]:
!{python} -m pip install graphviz;

# Análise das Stories

### Imagens de Fluxo

Gerando os arquivos de imagem para a visualização do fluxo das histórias. A célula abaixo gera uma imagem para cada arquivo de stories. 


In [9]:
from IPython.display import Image
from rasa_core.agent import Agent

from os import listdir
from os.path import isfile, join


agent = Agent('../../bot/domain.yml')

# Adds all stories files in a list
stories_files = [f for f in listdir("../../bot/data/stories") if isfile(join("../../bot/data/stories", f))]

# Generate the image for each file
for file in stories_files:
    agent.visualize('../../bot/data/stories/' + file,
                    './img/story_graph_' + file[:-3] + '.png',
                    max_history = 2)

print("Imagens salvas")

Processed Story Blocks: 100%|██████████| 11/11 [00:00<00:00, 34.04it/s, # trackers=1]


Imagens salvas


### Treinando modelo de dialogo

In [3]:
from rasa_core.policies import FallbackPolicy, KerasPolicy, MemoizationPolicy
from rasa_core.agent import Agent

# this will catch predictions the model isn't very certain about
# there is a threshold for the NLU predictions as well as the action predictions
fallback = FallbackPolicy(fallback_action_name="utter_unclear",
                          core_threshold=0.2,
                          nlu_threshold=0.6)

agent = Agent('../../bot/domain.yml', policies=[MemoizationPolicy(), KerasPolicy(), fallback])

# loading our neatly defined training dialogues
training_data = agent.load_data('../../bot/data/stories')

agent.train(
    training_data,
    validation_split=0.0,
    epochs=130
)

agent.persist('models/dialogue')

Processed Story Blocks: 100%|██████████| 125/125 [00:01<00:00, 68.75it/s, # trackers=19]
Processed actions: 15219it [00:32, 463.93it/s, # examples=15031]
INFO:rasa_core.policies.keras_policy:Fitting model with 15219 total samples and a validation split of 0.0


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
masking (Masking)            (None, 5, 259)            0         
_________________________________________________________________
lstm (LSTM)                  (None, 32)                37376     
_________________________________________________________________
dense (Dense)                (None, 125)               4125      
_________________________________________________________________
activation (Activation)      (None, 125)               0         
Total params: 41,501
Trainable params: 41,501
Non-trainable params: 0
_________________________________________________________________
Epoch 1/130
15219/15219 [==============================] - 5s 344us/step - loss: 3.0492 - acc: 0.3611
Epoch 2/130
15219/15219 [==============================] - 4s 290us/step - loss: 2.3451 - acc: 0.5153
Epoch 3/130
15219/15219 [==============================] - 5s 310u

15219/15219 [==============================] - 34s 2ms/step - loss: 0.0601 - acc: 0.9818
Epoch 73/130
15219/15219 [==============================] - 31s 2ms/step - loss: 0.0581 - acc: 0.9832
Epoch 74/130
15219/15219 [==============================] - 31s 2ms/step - loss: 0.0622 - acc: 0.9800: 1s - los
Epoch 75/130
15219/15219 [==============================] - 35s 2ms/step - loss: 0.0607 - acc: 0.9818
Epoch 76/130
15219/15219 [==============================] - 34s 2ms/step - loss: 0.0579 - acc: 0.9826
Epoch 77/130
15219/15219 [==============================] - 33s 2ms/step - loss: 0.0619 - acc: 0.9810
Epoch 78/130
15219/15219 [==============================] - 34s 2ms/step - loss: 0.0538 - acc: 0.9830
Epoch 79/130
15219/15219 [==============================] - 35s 2ms/step - loss: 0.0542 - acc: 0.9820
Epoch 80/130
15219/15219 [==============================] - 38s 3ms/step - loss: 0.0607 - acc: 0.9814
Epoch 81/130
15219/15219 [==============================] - 35s 2ms/step - loss: 0.04

INFO:rasa_core.policies.keras_policy:Done fitting keras policy model
INFO:rasa_core.agent:Model directory models/dialogue exists and contains old model files. All files will be overwritten.
INFO:rasa_core.agent:Persisted model to '/home/gabibs/Documentos/lappis/tais/notebooks/stories/models/dialogue'


### Starting up the bot (with NLU)

Now that we've trained the dialogue **and** language understanding models and saved them, we can start up an `Agent` which will handle conversations for us. 

In [12]:
from rasa_core.agent import Agent
agent = Agent.load('models/dialogue')

### Evaluation of the dialogue model
As with the NLU model, instead of just subjectively testing the model, we can also evaluate the model on a dataset. You'll be using the training data set again, but usually you'd use a test data set separate from the training data.

In [28]:
from rasa_core.evaluate import run_story_evaluation

#run_story_evaluation("../../bot/data/stories", "models/dialogue", 
#                     nlu_model_path=None, 
#                     max_stories=None, 
#                     out_file_plot="story_eval.pdf")

run_story_evaluation("../../bot/data/stories", agent, 2, "story_eval.pdf", "plot_eval.pdf", True)

Processed Story Blocks: 100%|██████████| 125/125 [00:03<00:00, 37.56it/s, # trackers=1]
INFO:rasa_core.evaluate:Evaluating 120 stories
Progress:
  0%|          | 0/120 [00:00<?, ?it/s]


Exception: BinarySingleStateFeaturizer was not prepared before encoding.